# SEIN power demand data

Power demand of the Peruvian national electric system (SEIN) registered in 30-minute periods and reported daily the next day of operations by the COES (Peru's system operator). This data is retrieved  from SCADA systems and published in Excel format.

Source: https://www.coes.org.pe/Portal/PostOperacion/Reportes/Ieod

Description of raw data extracted from daily reports:

- "datetime": Date and time in "yyyy-mm-dd hh:mm:ss" format.  
- "TOTAL GENERACIÓN COES": Total power dispatch  of COES's generators in MW.  
- "IMPORTACIÓN ECUADOR": Imports and exports from and to Ecuador in MW.  
- "TOTAL SEIN": Power demand of the national electric system (SEIN) in MW.   
- "TOTAL NO COES": Total power dispatch of non-COES's generators in MW.  

## Download data

Refactored code to download data of SEIN power demand in a single run.

In [14]:
import pandas as pd
import requests
from datetime import datetime
import os
import time
from openpyxl import load_workbook
import xlrd
import sagemaker
from sagemaker import get_execution_role

In [5]:
start_time = time.time()

session = requests.Session()

endpoint = 'www.coes.org.pe/Portal/browser/download?url=Post%20Operaci%C3%B3n/Reportes/IEOD'
url_list = [
    'https://{}/{}/{:02d}_{}/{:02d}/Anexo1_Resumen_{:02d}{:02d}',  # 2021
    'https://{}/{}/{:02d}%20{}/{:02d}/Anexo1_Resumen_{:02d}{:02d}',  # Jul 2019 - 2020
    'https://{}/{}/{:02d}%20{}/D%C3%ADa%20{:02d}/Anexo1_Resumen_{:02d}{:02d}',  # Jan-Jun 2019
    'https://{}/{}/{:02d}_{}/D%C3%ADa%20{:02d}/Anexo1_Resumen_{:02d}{:02d}',  # Dec 5 2017 - 2018
    'https://{}/{}/{:02d}_{}/D%C3%ADa%20{:02d}/Anexo_{:02d}{:02d}',  # <= 2017
    'https://{}/{}/{:02d}_{}/Dia%20{:02d}/Anexo_{:02d}{:02d}',
    'https://{}/{}/{:02d}_{}/D%C3%ACa%20{:02d}/Anexo1_Resumen_{:02d}{:02d}',
    'https://{}/{}/{:02d}%20{}/D%C3%ACa%20{:02d}/Anexo1_Resumen_{:02d}{:02d}',
    'https://{}/{}/{:02d}_{}/D%C3%ADa%20{:02d}/Anexo_{:02d}{:02d}_Resumen',
    'https://{}/{}/{:02d}%20{}/Dia%20{:02d}/Anexo1_Resumen_{:02d}{:02d}',
]

start_date = datetime(2016, 1, 1)
end_date = datetime(2023, 3, 31)

extensions = ['.xlsx', '.xls']

header_names = ["HORA", "TOTAL GENERACIÓN COES", "IMPORTACIÓN  ECUADOR",
                "TOTAL SEIN", "TOTAL\nNO COES"]
sheet_name = 'DESPACHO_EJECUTADO'

start_row = 6
end_row = 53

month_dict = {
    'January': 'Enero',
    'February': 'Febrero',
    'March': 'Marzo',
    'April': 'Abril',
    'May': 'Mayo',
    'June': 'Junio',
    'July': 'Julio',
    'August': 'Agosto',
    'September': 'Setiembre',
    'October': 'Octubre',
    'November': 'Noviembre',
    'December': 'Diciembre'
}

df_list = []

if not os.path.exists("./sein_demand"):
    os.makedirs("./sein_demand")

count_files = 0

for date in pd.date_range(start_date, end_date):
    year, month, day = date.year, date.month, date.day
    month_name = month_dict[date.strftime('%B')]

    file_path = ""

    for ext in extensions:
        file_path = f'./sein_demand/{year}_{month:02d}_{day:02d}{ext}'

        for url in url_list:
            file_url = url.format(endpoint, year, month, month_name,
                                  day, day, month) + ext

            try:
                response = session.get(file_url, stream=True)
                response.raise_for_status()

                with open(file_path, 'wb') as f:
                    f.write(response.content)

                if ext == ".xlsx":
                    load_workbook(file_path)
                else:
                    xlrd.open_workbook(file_path)

                count_files += 1
                break

            except requests.exceptions.RequestException as e:
                print("Error during request:", e, date.date())

            except Exception as e:
                # print("Error:", e, file_path)
                os.remove(file_path)
        else:
            continue
        break

    df = pd.DataFrame(columns=["datetime"])

    try:
        header_row = pd.read_excel(file_path, sheet_name=sheet_name,
                                   header=None, nrows=1, skiprows=start_row-2)
        header_values = header_row.loc[0, :].tolist()
        main_index = header_values.index(header_names[1])
        last_index = header_values.index(header_names[4])
        indices = [0, main_index, main_index+1, main_index+2, last_index]

        df = pd.read_excel(file_path, sheet_name=sheet_name,
                           header=start_row-2, usecols=indices,
                           nrows=end_row-start_row+1, skiprows=[start_row-1])
        df.columns = header_names

    except Exception as e:
        print("Error reading file:", e, date.date())

    if not df.empty:
        df.insert(0, 'datetime', pd.to_datetime(
            f'{year}-{month:02d}-{day:02d} ' + df['HORA'].astype(str)))

        mask = df['HORA'] == '00:00'
        df.loc[mask, 'datetime'] = df.loc[mask, 'datetime'] + pd.Timedelta(days=1)

        df = df.drop(columns=['HORA'])
        df.rename(columns={df.columns[2]: 'IMPORTACIÓN  ECUADOR'}, inplace=True)

    df_list.append(df)

merged_df = pd.concat(df_list, ignore_index=True)

date_range = pd.date_range(start_date, end_date + pd.Timedelta(days=1), freq='30min')
merged_df = merged_df.set_index('datetime').reindex(date_range).reset_index()
merged_df.rename(columns={'index': 'datetime'}, inplace=True)
merged_df = merged_df.drop(index=0)

merged_df.to_csv('./sein_demand.csv', index=False)

end_time = time.time()
time_taken = end_time - start_time
print(f'Downloaded files: {count_files} files')
print(f'Time taken: {time_taken} seconds')

Error reading file: [Errno 2] No such file or directory: './sein_demand/2019_07_31.xls' 2019-07-31


/opt/conda/lib/python3.7/site-packages/openpyxl/reader/drawings.py:59: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)


Error reading file: [Errno 2] No such file or directory: './sein_demand/2020_11_04.xls' 2020-11-04
Error reading file: [Errno 2] No such file or directory: './sein_demand/2020_12_25.xls' 2020-12-25
Downloaded files: 2644 files
Time taken: 16804.24077439308 seconds


In [15]:
merged_df = pd.read_csv('./sein_demand.csv')
merged_df

,datetime,TOTAL GENERACIÓN COES,IMPORTACIÓN ECUADOR,TOTAL SEIN,TOTAL\nNO COES
0,2016-01-01 00:30:00,5028.33192,41.77,4986.56192,22.62600
1,2016-01-01 01:00:00,4948.64387,41.23,4907.41387,23.63700
2,2016-01-01 01:30:00,4817.33694,38.79,4778.54694,23.00700
3,2016-01-01 02:00:00,4702.32782,40.13,4662.19782,22.82900
4,2016-01-01 02:30:00,4562.66431,39.50,4523.16431,22.84000
...,...,...,...,...,...
127051,2023-03-31 22:00:00,7225.40000,0.00,7225.40000,92.63953
127052,2023-03-31 22:30:00,7077.20000,0.00,7077.20000,84.29747
127053,2023-03-31 23:00:00,6943.00000,0.00,6943.00000,88.19350
127054,2023-03-31 23:30:00,6715.00000,0.00,6715.00000,93.18016


### Load data into S3

In [ ]:
os.environ['PROJECT_BUCKET'] = ""

In [10]:
session = sagemaker.Session()
bucket = os.getenv("PROJECT_BUCKET")
region = session.boto_region_name
role = get_execution_role()

print("AWS Region: {}".format(region))

AWS Region: us-east-1


In [ ]:
!aws s3 sync ./sein_demand s3://{bucket}/data/sein_demand/

In [ ]:
!aws s3 cp ./sein_demand.csv s3://{bucket}/data/